# Kasus
Misalkan ada suatu Rumah Sakit kecil yang memiliki kondisi sebagai berikut

Suatu Rumah Sakit memiliki 3 orang Perawat (Annisa, Balqis, Cantika).
RS buka selama 24 jam, setiap jam harus ada seorang perawat yang berjaga.
Pembagian jaga dibagi menjadi 3 shift yaitu Pagi, Siang, Malam.
RS buka hanya 3 hari, yaitu Senin, Selasa, Rabu.

Karena beberapa kondisi pembagian jaga sebagai berikut
1.   Annisa jaga sebanyak 3 kali dalam sepekan
2.   Balqis jaga 2 kali dalam sepekan
3.   Cantika jaga 4 kali dalam sepekan

Kendala matematis dari masalah jadwal di atas adalah sebagai berikut :

3. Untuk setiap shift, hanya ada 1 perawat yang berjaga
4. Setiap Perawat tidak diperbolehkan jaga 2 kali secara berurutan

Untuk setiap perawat, berjaga sesuai penugasan
5. Annisa jaga sebanyak 3 kali dalam sepekan
6. Balqis jaga sebanyak 2 kali dalam sepekan
7. Cantika jaga sebanyak 4 kali dalam sepekan

Tambahan kendala lain
8. Balqis hanya boleh jaga pada shift pagi.
9. Annisa hanya boleh berjaga 1 kali dalam sehari


# Model Matematika

Misalkan $x_p._h._s$ menyatakan Perawat ke-$p$ berjaga pada hari ke-$h$ di Shift ke-$s$

$p=\{1,2,3\}$ Perawat ke-1 (Annisa), ke-2 (Balqis), ke-3 (Cantika)

$h=\{1,2,3\}$  Hari ke-1 (Senin), ke-2 (Selasa), ke-3 (Rabu)

$s=\{1,2,3\}$ Shift ke-1 (Pagi), ke-2 (Siang), ke-3 (Malam)

$x_p._h._s = 1 $, apabila Perawat ke-$p$ **BERJAGA** pada hari ke-$h$ di Shift ke-$s$

$x_p._h._s = 0 $, apabila Perawat ke-$p$ **TIDAK BERJAGA** pada hari ke-$h$ di Shift ke-$s$

$np$: banyaknya Perawat
$nh$: banyaknya Hari
$ns$ : banyaknya Sesi

$P$ : Himpunan Perawat
$H$ : Himpunan Hari
$S$ : Himpunan Shift




In [1]:
!pip install mip #Apabila library mip sudah terinstall, berikan comment (tanda #) sebelum !pip
import mip 
import pandas as pd

     |████████████████████████████████| 48.0 MB 47 kB/s 


In [2]:
Perawat=["Annisa","Balqis","Cantika"] #Daftar Perawat
Hari=["Senin","Selasa","Rabu"] #Daftar Hari
Shift=["Pagi","Siang","Malam"] #Daftar Shift
Tugas=[3,2,4] #Penugasan masing-masing perawat

In [3]:
np=len(Perawat);nh=len(Hari);ns=len(Shift)

m = mip.Model("Contoh Jadwal Perawat")
cost = [[[1 for p in range(np)] for h in range(nh)] for s in range(ns)]
x = [[[m.add_var(var_type=mip.BINARY) for p in range(np)] for h in range(nh)] for s in range(ns)]

m.objective = mip.minimize(mip.xsum(cost[p][h][s]*x[p][h][s] for p in range(np) for h in range(nh) for s in range(ns)))
#model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

#kendala 0 : Banyaknya shift jaga yang harus diisi adalah 3 hari kali 3 shift = 9 shift
m += mip.xsum(x[p][h][s] for p in range(np) for h in range(nh) for s in range(ns)) == nh*ns
#kendala 1 : Untuk setiap shift, hanya ada 1 perawat yang berjaga
for h in range (nh): 
  for s in range (ns):
    m += mip.xsum(x[p][h][s] for p in range(np)) == 1

#kendala 2 : Setiap Perawat tidak diperbolehkan jaga 2 kali secara berurutan
for p in range (np): 
  for h in range (nh):
    for s in range (ns-1):
      m += x[p][h][s]+x[p][h][s+1] <= 1 
  for h in range (nh-1):
      m += x[p][h][ns-1]+x[p][h+1][0] <= 1


#kendala 3,4,5 : Annisa, Balqis dan Cantika jaga sesuai penugasan
for p in range(np):
  m += mip.xsum(x[p][h][s] for h in range(nh) for s in range(ns)) == Tugas[p]  

#kendala 6 : Balqis (p=2) hanya boleh jaga pada shift pagi.
p=2
m += mip.xsum( x[p-1][h][0] for h in range(nh) ) == 2

#kendala 7 : Annisa (p=1) hanya boleh berjaga 1 kali dalam sehari
p=1
for h in range(nh):
  m += mip.xsum(x[p-1][h][s] for s in range(ns)) <= 1

m.optimize()

print("Jadwal Terpilih :")
jadwal1=[[1 for h in range(nh)] for s in range(ns)] #membuat list untuk menyimpan jadwal
for p in range (np): 
  for h in range (nh): 
    for s in range (ns):
      if(x[p][h][s].x>0.5): print("x[",p+1,",",h+1,",",s+1,"]=",Perawat[p],"-",Hari[h],"-",Shift[s]);jadwal1[s][h]=Perawat[p]

#Mengeluarkan Jadwal dalam bentuk tabel
print("\nKeluaran jadwal dalam bentuk tabel :")
tabel1 = pd.DataFrame(jadwal1, columns=Hari, index=Shift)
print(tabel1)

Jadwal Terpilih :
x[ 1 , 1 , 2 ]= Annisa - Senin - Siang
x[ 1 , 2 , 2 ]= Annisa - Selasa - Siang
x[ 1 , 3 , 3 ]= Annisa - Rabu - Malam
x[ 2 , 2 , 1 ]= Balqis - Selasa - Pagi
x[ 2 , 3 , 1 ]= Balqis - Rabu - Pagi
x[ 3 , 1 , 1 ]= Cantika - Senin - Pagi
x[ 3 , 1 , 3 ]= Cantika - Senin - Malam
x[ 3 , 2 , 3 ]= Cantika - Selasa - Malam
x[ 3 , 3 , 2 ]= Cantika - Rabu - Siang

Keluaran jadwal dalam bentuk tabel :
         Senin   Selasa     Rabu
Pagi   Cantika   Balqis   Balqis
Siang   Annisa   Annisa  Cantika
Malam  Cantika  Cantika   Annisa


# Tambahan Kendala Khusus
Andaikan terdapat kendala khusus sebagai berikut

Cantika diusahakan tidak jaga pada Senin Pagi ( x3,1,1 )
Cantika diusahakan jaga pada Selasa Pagi ( x3,2,1 )

Kita bisa memberikan konstanta pengali pada variabel keputusa yang berperan sebagai "biaya".
Pada dasarnya konstanta "biaya" bernilai 1. Bisanya konstanta ini dinamakan cost, dengan variabel  c .

Bila diinginkan suatu variabel keputusan diusahakan terpilih, maka berikan "biaya" yang "lebih murah", bernilai  c<1 . Contohnya  c=0.5 
Bila diinginkan suatu variabel keputusan diusahakan TIDAK terpilih, maka berikan "biaya" yang "lebih mahal", bernilai  c>1 . Contohnya  c=2 
Fungsi objektif yang dimiliki adalah meminimumkan, sehingga konstanta yang bernilai lebih kecil akan lebih dipilih.

In [4]:
#Modifikasi Tambahan Kendala Khusus (1)
np=len(Perawat);nh=len(Hari);ns=len(Shift)

m = mip.Model("Contoh Jadwal Perawat")
cost = [[[1 for p in range(np)] for h in range(nh)] for s in range(ns)]
#Modifikasi Tambahan Kendala Khusus (1) :
cost[2][0][0]=2 #Cantika diusahakan tidak jaga pada Senin Pagi y_{18}=x_{3,1,1}
cost[2][1][0]=0.5 # Cantika diusahakan jaga pada Selasa Pagi y_{21}=x_{3,2,1}
x = [[[m.add_var(var_type=mip.BINARY) for p in range(np)] for h in range(nh)] for s in range(ns)]

m.objective = mip.minimize(mip.xsum(cost[p][h][s]*x[p][h][s] for p in range(np) for h in range(nh) for s in range(ns)))
#model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

#kendala 0 : Banyaknya shift jaga yang harus diisi adalah 3 hari kali 3 shift = 9 shift
m += mip.xsum(x[p][h][s] for p in range(np) for h in range(nh) for s in range(ns)) == nh*ns
#kendala 1 : Untuk setiap shift, hanya ada 1 perawat yang berjaga
for h in range (nh): 
  for s in range (ns):
    m += mip.xsum(x[p][h][s] for p in range(np)) == 1

#kendala 2 : Setiap Perawat tidak diperbolehkan jaga 2 kali secara berurutan
for p in range (np): 
  for h in range (nh):
    for s in range (ns-1):
      m += x[p][h][s]+x[p][h][s+1] <= 1 
  for h in range (nh-1):
      m += x[p][h][ns-1]+x[p][h+1][0] <= 1


#kendala 3,4,5 : Annisa, Balqis dan Cantika jaga sesuai penugasan
tugas=[3,2,4]
for p in range(np):
  m += mip.xsum(x[p][h][s] for h in range(nh) for s in range(ns)) == tugas[p]  

#kendala 6 : Balqis (p=2) hanya boleh jaga pada shift pagi.
p=2
m += mip.xsum( x[p-1][h][0] for h in range(nh) ) == 2

#kendala 7 : Annisa (p=1) hanya boleh berjaga 1 kali dalam sehari
p=1
for h in range(nh):
  m += mip.xsum(x[p-1][h][s] for s in range(ns)) <= 1

m.optimize()

print("Jadwal Terpilih :")
jadwal2=[[1 for h in range(nh)] for s in range(ns)] #membuat list untuk menyimpan jadwal
for p in range (np): 
  for h in range (nh): 
    for s in range (ns):
      if(x[p][h][s].x>0.5): print("x[",p+1,",",h+1,",",s+1,"]=",Perawat[p],"-",Hari[h],"-",Shift[s]);jadwal2[s][h]=Perawat[p]

#Mengeluarkan Jadwal dalam bentuk tabel
print("\nKeluaran jadwal dalam bentuk tabel :")
tabel2 = pd.DataFrame(jadwal2, columns=Hari, index=Shift)
print(tabel2)

Jadwal Terpilih :
x[ 1 , 1 , 3 ]= Annisa - Senin - Malam
x[ 1 , 2 , 2 ]= Annisa - Selasa - Siang
x[ 1 , 3 , 3 ]= Annisa - Rabu - Malam
x[ 2 , 1 , 1 ]= Balqis - Senin - Pagi
x[ 2 , 3 , 1 ]= Balqis - Rabu - Pagi
x[ 3 , 1 , 2 ]= Cantika - Senin - Siang
x[ 3 , 2 , 1 ]= Cantika - Selasa - Pagi
x[ 3 , 2 , 3 ]= Cantika - Selasa - Malam
x[ 3 , 3 , 2 ]= Cantika - Rabu - Siang

Keluaran jadwal dalam bentuk tabel :
         Senin   Selasa     Rabu
Pagi    Balqis  Cantika   Balqis
Siang  Cantika   Annisa  Cantika
Malam   Annisa  Cantika   Annisa


# Perbandingan

In [5]:
print("Bandingkan :")
print("\nJadwal Pertama :")
print(tabel1)
print("\nJadwal Kedua dengan tambahan kendala khusus (1) :")
print(tabel2)

Bandingkan :

Jadwal Pertama :
         Senin   Selasa     Rabu
Pagi   Cantika   Balqis   Balqis
Siang   Annisa   Annisa  Cantika
Malam  Cantika  Cantika   Annisa

Jadwal Kedua dengan tambahan kendala khusus (1) :
         Senin   Selasa     Rabu
Pagi    Balqis  Cantika   Balqis
Siang  Cantika   Annisa  Cantika
Malam   Annisa  Cantika   Annisa
